**Linear/Polynomial Regression Analysis of Temperature in Lake Tahoe and Snowpack**

In [2]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [15]:
ds1 = xr.open_dataset("test3.nc")
ds1

<xarray.Dataset> Size: 4MB
Dimensions:     (valid_time: 1030, latitude: 19, longitude: 23)
Coordinates:
    number      int64 8B ...
  * valid_time  (valid_time) datetime64[ns] 8kB 1940-01-01T06:00:00 ... 2025-...
  * latitude    (latitude) float64 152B 40.0 39.75 39.5 ... 36.0 35.75 35.5
  * longitude   (longitude) float64 184B -119.5 -119.2 -119.0 ... -114.2 -114.0
    expver      (valid_time) <U4 16kB ...
Data variables:
    tp          (valid_time, latitude, longitude) float32 2MB ...
    sf          (valid_time, latitude, longitude) float32 2MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-11-30T02:45 GRIB to CDM+CF via cfgrib-0.9.1...

In [16]:
ds2 = xr.open_dataset("test4.nc")
ds2

<xarray.Dataset> Size: 5MB
Dimensions:     (valid_time: 1030, latitude: 19, longitude: 23)
Coordinates:
    number      int64 8B ...
  * valid_time  (valid_time) datetime64[ns] 8kB 1940-01-01 ... 2025-10-01
  * latitude    (latitude) float64 152B 40.0 39.75 39.5 ... 36.0 35.75 35.5
  * longitude   (longitude) float64 184B -119.5 -119.2 -119.0 ... -114.2 -114.0
    expver      (valid_time) <U4 16kB ...
Data variables:
    t2m         (valid_time, latitude, longitude) float32 2MB ...
    tcrw        (valid_time, latitude, longitude) float32 2MB ...
    tcsw        (valid_time, latitude, longitude) float32 2MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-11-30T02:45 GRIB to CDM+CF via cfgrib-0.9.1...

In [17]:
# Need to standardize the time columns between both datasets
print(ds1.valid_time)
print(ds2.valid_time)

<xarray.DataArray 'valid_time' (valid_time: 1030)> Size: 8kB
array(['1940-01-01T06:00:00.000000000', '1940-02-01T06:00:00.000000000',
       '1940-03-01T06:00:00.000000000', ..., '2025-08-01T06:00:00.000000000',
       '2025-09-01T06:00:00.000000000', '2025-10-01T06:00:00.000000000'],
      shape=(1030,), dtype='datetime64[ns]')
Coordinates:
    number      int64 8B ...
  * valid_time  (valid_time) datetime64[ns] 8kB 1940-01-01T06:00:00 ... 2025-...
    expver      (valid_time) <U4 16kB ...
Attributes:
    long_name:      time
    standard_name:  time
<xarray.DataArray 'valid_time' (valid_time: 1030)> Size: 8kB
array(['1940-01-01T00:00:00.000000000', '1940-02-01T00:00:00.000000000',
       '1940-03-01T00:00:00.000000000', ..., '2025-08-01T00:00:00.000000000',
       '2025-09-01T00:00:00.000000000', '2025-10-01T00:00:00.000000000'],
      shape=(1030,), dtype='datetime64[ns]')
Coordinates:
    number      int64 8B ...
  * valid_time  (valid_time) datetime64[ns] 8kB 1940-01-01 ... 2025-1

In [18]:
ds1['valid_time'] = ds1.valid_time.dt.floor('D')
ds2['valid_time'] = ds2.valid_time.dt.floor('D')

In [19]:
ds = xr.merge([ds1, ds2])
ds

/tmp/ipykernel_932/3798341914.py:1: FutureWarning: In a future version of xarray the default value for compat will change from compat='no_conflicts' to compat='override'. This is likely to lead to different results when combining overlapping variables with the same name. To opt in to new defaults and get rid of these warnings now use `set_options(use_new_combine_kwarg_defaults=True) or set compat explicitly.
  ds = xr.merge([ds1, ds2])
/tmp/ipykernel_932/3798341914.py:1: FutureWarning: In a future version of xarray the default value for compat will change from compat='no_conflicts' to compat='override'. This is likely to lead to different results when combining overlapping variables with the same name. To opt in to new defaults and get rid of these warnings now use `set_options(use_new_combine_kwarg_defaults=True) or set compat explicitly.
  ds = xr.merge([ds1, ds2])


<xarray.Dataset> Size: 9MB
Dimensions:     (valid_time: 1030, latitude: 19, longitude: 23)
Coordinates:
    number      int64 8B 0
  * valid_time  (valid_time) datetime64[ns] 8kB 1940-01-01 ... 2025-10-01
  * latitude    (latitude) float64 152B 40.0 39.75 39.5 ... 36.0 35.75 35.5
  * longitude   (longitude) float64 184B -119.5 -119.2 -119.0 ... -114.2 -114.0
    expver      (valid_time) <U4 16kB '0001' '0001' '0001' ... '0001' '0005'
Data variables:
    tp          (valid_time, latitude, longitude) float32 2MB ...
    sf          (valid_time, latitude, longitude) float32 2MB ...
    t2m         (valid_time, latitude, longitude) float32 2MB ...
    tcrw        (valid_time, latitude, longitude) float32 2MB ...
    tcsw        (valid_time, latitude, longitude) float32 2MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-11-30T02:45 GRIB to CDM+CF via cfgrib-0.9.1...

In [20]:
print(ds.data_vars)

Data variables:
    tp       (valid_time, latitude, longitude) float32 2MB ...
    sf       (valid_time, latitude, longitude) float32 2MB ...
    t2m      (valid_time, latitude, longitude) float32 2MB ...
    tcrw     (valid_time, latitude, longitude) float32 2MB ...
    tcsw     (valid_time, latitude, longitude) float32 2MB ...


In [26]:
ds['t2m'].to_series().describe()

count    450110.000000
mean        285.010345
std          10.182698
min         256.652832
25%         276.943420
50%         284.813721
75%         293.428223
max         310.864990
Name: t2m, dtype: float64

In [31]:
annual_snowfall = ds['sf'].resample(valid_time='YS').sum()


In [ ]:
# First I want to organize my dataset and do some exploratory data analysis 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [ ]:
# We need to establish the features matrix and target array
X_dsfeatures = ds.drop('R (mm/hr)', axis=1)
X_dsfeatures.shape

In [ ]:
y_dstarget = ds['R (mm/hr)']
y_dstarget.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_dsfeatures, y_dstarget, random_state=0,
                                  train_size=0.7)

In [ ]:
# train model on training dataset
model = LinearRegression(fit_intercept=True)
model.fit(X_train, y_train)

In [ ]:
# Predict results based on the each dataset
y_pred_from_train = model.predict(X_train)
y_pred_from_test = model.predict(X_test)